# **Homework 2-Steam Reviews 2021**
## **Research Questions**

### [RQ1] Exploratory data analysis


### [RQ2]

In [ ]:
import pandas as pd
dataset = pd.read_csv('./steam_reviews.csv', header='infer', nrows=1000)

In [ ]:
dataset.groupby('app_name').review.count()

### [RQ3]

In [ ]:
import pandas as pd

In [ ]:
def dateparse(time_in_secs):
    return pd.to_datetime(time_in_secs, unit='s')

In [ ]:
steam_small = pd.read_csv('C:/Users/Filippo/Desktop/università/ADM/steam_reviews.csv',header='infer',nrows=1000000,parse_dates=['timestamp_created','timestamp_updated', 'author.last_played'],date_parser=dateparse)

In [ ]:
creation=steam_small.timestamp_created.dt.time

In [ ]:
creation.value_counts()

### [RQ4]